In [1]:
import pandas as pd
df = pd.read_csv(r"permno.csv")

,PERMNO,NAMEDT,EXCHCD,TICKER,COMNAM,PERMCO,BEGEXCHDATE,ENDEXCHDATE
0,10000,19860107,3,OMFGA,OPTIMUM MANUFACTURING INC,7952,19860131,19870630
1,10001,19931122,3,EWST,ENERGY WEST INC,7953,19860131,20170831
2,10002,20020515,3,BTFG,BANCTRUST FINANCIAL GROUP INC,7954,19860131,20130228
3,10003,19860114,3,GCBK,GREAT COUNTRY BK ASONIA CT,7957,19860131,19951229
4,10005,19860115,3,WERC,WESTERN ENERGY RESOURCES INC,7961,19860131,19910731
...,...,...,...,...,...,...,...,...
15745,93279,19850918,3,ZABS,ZABS BACKYARD HOTS INC,7736,19850930,19870331
15746,93287,19850318,3,ZNTL,ZEHNTEL INC,7466,19850329,19871231
15747,93308,19850410,3,ZETA,ZETA LABS INC,7490,19850430,19860331
15748,93316,19930702,3,ZING,ZING TECHNOLOGIES INC,7469,19850329,20000331


I write the PERMNOs to a text file then input into CRSP monthly to get exchange data, because a stock may switch exchanges and I'm only interested in those listed on the NASDAQ- and CCM only gives the last exchange the stock was listed on.

In [3]:
df = pd.read_csv(r"crsp data.csv")
df = df.rename(columns={"date": "Date", "EXCHCD": "Exchange Code",
                          "TICKER":"Ticker","COMNAM":"Company",
                          "DIVAMT":"Dividend","PRC":"Price","VOL":"Volume","SHROUT":"Shares Outstanding"
                         })
df = df[df["Price"].notna()]
df = df[df["Exchange Code"]==3]

In [4]:
df["Dividend"] = df["Dividend"].fillna(0)
# CRSP's hyphenated price values means there was no close price, but instead a bid/ask average.
df["Price"] = df["Price"].abs()
df["Date"] = pd.to_datetime(df["Date"], format='%Y%m%d', errors='ignore')
df= df.sort_values(by=['PERMNO','Date','Dividend']) 
#things are mostly ordered, but I want to sort by dividend too in order to drop duplicates and keep the largest dividend entry.
df = df.drop_duplicates(subset = ["PERMNO","Date"], keep ="last").reset_index(drop = True)

In [6]:
permno = open("checked_permno.txt","w")
for i in df["PERMNO"].unique():
     permno.write(str(i)+"\n")
permno.close()

17841/17887 stocks do have the NASDAQ as their exchange code (=3) at some point. This is the checked_permno which I then upload to CCM.